# Домашнее задание 4

In [2]:
 !pip install kaggle-environments
# !pip install termcolor pygame chex

In [3]:
# Подключаем нужные библиотеки
from kaggle_environments import make, evaluate
import numpy as np
import random
import pandas as pd


Loading environment lux_ai_s3 failed: No module named 'gymnax'


In [4]:
# Обозначения для ходов:
# 0 - камень
# 1 - бумага
# 2 - ножницы

# Создаем окружение для игры с заданным количеством шагов (ходов)
env = make("rps", configuration={"episodeSteps": 100})

### Реализуем простейших ботов для игры в камень-ножницы-бумага

Агент 001: Всегда выбирает камень

In [5]:
%%writefile mason.py
# Всегда выбирает камень
def mason(observation, configuration):
    return 0

Overwriting mason.py


Агент 002: Всегда выбирает ножницы

In [6]:
%%writefile taylor.py
# Всегда выбирает ножницы
def taylor(observation, configuration):
    return 2

Overwriting taylor.py


Агент 003: Всегда выбирает бумагу

In [7]:
%%writefile writer.py
# Всегда выбирает бумагу
def writer(observation, configuration):
    return 1

Overwriting writer.py


Агент 004: Случайный выбор из 3 значений

In [8]:
%%writefile rnd_agent.py
import random
# Случайный выбор из 3 значений
def rnd_agent(observation, configuration):
    return random.randint(0, 2)

Overwriting rnd_agent.py


Агент 005: Случайный выбор из 2 значений: камень или бумага

In [9]:
%%writefile rnd_rp_agent.py
import random
# Случайный выбор из 2 значений: камень или бумага
def rnd_rp_agent(observation, configuration):
    return random.randint(0, 1)

Overwriting rnd_rp_agent.py


Агент 006: Случайный выбор из 2 значений: бумага или ножницы

In [10]:
%%writefile rnd_ps_agent.py
import random
# Случайный выбор из 2 значений: бумага или ножницы
def rnd_ps_agent(observation, configuration):
    return random.randint(1, 2)

Overwriting rnd_ps_agent.py


Агент 007: Случайный выбор из 2 значений: камень или ножницы

In [11]:
%%writefile rnd_rs_agent.py
import random
# Случайный выбор из 2 значений: камень или ножницы
def rnd_rs_agent(observation, configuration):
    x = random.randint(0, 1)
    if x==0:
        return 0
    else:
        return 2

Overwriting rnd_rs_agent.py


Агент 008: Выдает результат по циклу rock -> paper -> scissors

In [12]:
%%writefile r_p_s_agent.py
import random
# Выдает результат по циклу rock -> paper -> scissors
last_action = None
def r_p_s_agent(observation, configuration):
    global last_action
    if observation.step == 0:
        # если это первый ход
        last_action = random.randint(0, 2)
    else:
        last_action = (last_action + 1) % 3
    return last_action
    

Overwriting r_p_s_agent.py


Агент 009: Выдает результат по циклу scissors -> paper -> rock

In [13]:
%%writefile s_p_r_agent.py
import random
# Выдает результат по циклу scissors -> paper -> rock
last_action = None
def s_p_r_agent(observation, configuration):
    global last_action
    if observation.step == 0:
        # если это первый ход
        last_action = random.randint(0, 2)
    else:
        last_action = (last_action + 2) % 3
    return last_action

Overwriting s_p_r_agent.py


Агент 010: Копирует последний ход противника

In [14]:
%%writefile mimic.py
# Агент, который копирует последний ход оппонента
import random

def mimic(observation, configuration):
    if observation.step == 0:
        return random.randint(1, 2) # случайный ход на первом шаге
    else:
        return observation.lastOpponentAction  # копирует предыдущий ход противника

Overwriting mimic.py


Агент 011: Смещается на +1 от последнего хода противника

In [15]:
%%writefile mimic_forward.py
# Агент, который смещается на +1 от последнего хода противника
import random

def mimic_forward(observation, configuration):
    if observation.step == 0:
        return random.randint(1, 2) # случайный ход на первом шаге
    else:
        return (observation.lastOpponentAction + 1) % 3  # смещение на +1 от последнего хода противника

Overwriting mimic_forward.py


Агент 012: Смещается на -1 от последнего хода противника

In [16]:
%%writefile mimic_backward.py
# Агент, который смещается на +1 от последнего хода противника
import random

def mimic_backward(observation, configuration):
    if observation.step == 0:
        return random.randint(1, 2) # случайный ход на первом шаге
    else:
        return (observation.lastOpponentAction + 2) % 3  # смещение на -1 от последнего хода противника

Overwriting mimic_backward.py


### Сравниваем результативность наших агентов в турнире
(каждый агент соревнуется с каждым, победитель определяется как агент, победивший наибольшее число других агентов)

In [18]:
# Турнир между агентами
agents = ['mason.py', 'taylor.py', 'writer.py',
          'rnd_agent.py', 'rnd_rp_agent.py', 'rnd_ps_agent.py',
         'rnd_rs_agent.py', 'r_p_s_agent.py', 's_p_r_agent.py',
         'mimic.py', 'mimic_forward.py', 'mimic_backward.py']

n_agents = len(agents)

# Инициализируем матрицу результатов
scores = np.zeros((n_agents, n_agents))

# Играем каждый агент против каждого
for i in range(n_agents):
    for j in range(i + 1, n_agents):
        result = evaluate("rps", [agents[i], agents[j]], configuration={"episodeSteps": 100, "tieRewardThreshold":1}, num_episodes=1)
        scores[i][j] = result[0][0]  # сохраняем результат для агента i
        scores[j][i] = result[0][1]  # сохраняем результат для агента j

# Определяем победителя
vic = []
for i in range(n_agents):
    cnt_vic = sum(1 for j in range(n_agents) if scores[i][j] > 0)  # считаем победы агента
    vic.append(cnt_vic)

# Определяем индексы победителей
i_victors = [i for i, v in enumerate(vic) if v == max(vic)]
if len(i_victors) > 1:
    print("The winners are", '\n', *[agents[i] for i in i_victors], "!")
else:
    print("The winner is", agents[i_victors[0]], "!")

The winner is mimic_forward.py !


### В нашем случае победил агент, реализующий модель копирования действий противника со смещением на +1.

Требуется учитывать, что результат при повторном запуске может меняться, поскольку многие агенты работают на базе случайных чисел.